In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [2]:
TRAIN_DIR = os.path.join(os.getcwd(), "Face_Detection", "images", "train")
TEST_DIR = os.path.join(os.getcwd(), "Face_Detection", "images", "test")

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [4]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
print(train)

                                                   image     label
0      C:\Users\fs678\Desktop\C++ programmes\Face_Det...     angry
1      C:\Users\fs678\Desktop\C++ programmes\Face_Det...     angry
2      C:\Users\fs678\Desktop\C++ programmes\Face_Det...     angry
3      C:\Users\fs678\Desktop\C++ programmes\Face_Det...     angry
4      C:\Users\fs678\Desktop\C++ programmes\Face_Det...     angry
...                                                  ...       ...
28816  C:\Users\fs678\Desktop\C++ programmes\Face_Det...  surprise
28817  C:\Users\fs678\Desktop\C++ programmes\Face_Det...  surprise
28818  C:\Users\fs678\Desktop\C++ programmes\Face_Det...  surprise
28819  C:\Users\fs678\Desktop\C++ programmes\Face_Det...  surprise
28820  C:\Users\fs678\Desktop\C++ programmes\Face_Det...  surprise

[28821 rows x 2 columns]


In [6]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [7]:
print(test)
print(test['image'])

                                                  image     label
0     C:\Users\fs678\Desktop\C++ programmes\Face_Det...     angry
1     C:\Users\fs678\Desktop\C++ programmes\Face_Det...     angry
2     C:\Users\fs678\Desktop\C++ programmes\Face_Det...     angry
3     C:\Users\fs678\Desktop\C++ programmes\Face_Det...     angry
4     C:\Users\fs678\Desktop\C++ programmes\Face_Det...     angry
...                                                 ...       ...
7061  C:\Users\fs678\Desktop\C++ programmes\Face_Det...  surprise
7062  C:\Users\fs678\Desktop\C++ programmes\Face_Det...  surprise
7063  C:\Users\fs678\Desktop\C++ programmes\Face_Det...  surprise
7064  C:\Users\fs678\Desktop\C++ programmes\Face_Det...  surprise
7065  C:\Users\fs678\Desktop\C++ programmes\Face_Det...  surprise

[7066 rows x 2 columns]
0       C:\Users\fs678\Desktop\C++ programmes\Face_Det...
1       C:\Users\fs678\Desktop\C++ programmes\Face_Det...
2       C:\Users\fs678\Desktop\C++ programmes\Face_Det...
3       C

In [8]:
from tqdm.notebook import tqdm
!pip install Pillow


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale = True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [10]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Users\fs678\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [11]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [12]:
x_train = train_features/255.0
x_test = test_features/255.0

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [15]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [16]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [17]:
model = Sequential()

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(7, activation='softmax'))


C:\Users\fs678\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [19]:
model.fit(x= x_train, y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 196s 860ms/step - accuracy: 0.2338 - loss: 1.8374 - val_accuracy: 0.2583 - val_loss: 1.8110
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 194s 859ms/step - accuracy: 0.2510 - loss: 1.8146 - val_accuracy: 0.2644 - val_loss: 1.7962
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 197s 872ms/step - accuracy: 0.2596 - loss: 1.7825 - val_accuracy: 0.3166 - val_loss: 1.6684
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 193s 855ms/step - accuracy: 0.3257 - loss: 1.6781 - val_accuracy: 0.4278 - val_loss: 1.5029
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 195s 861ms/step - accuracy: 0.3879 - loss: 1.5445 - val_accuracy: 0.4641 - val_loss: 1.4018
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 204s 901ms/step - accuracy: 0.4237 - loss: 1.4747 - val_accuracy: 0.4853 - val_loss: 1.3365
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 195s 864ms/step - accuracy: 0.4551 - loss: 1.4189 - val_accuracy: 0.5112 - val_loss: 1.2793
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 192s 849ms/step - accuracy: 

In [20]:
model_json = model.to_json()
with open("facedetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("facedetector.h5")

In [21]:
from keras.models import model_from_json

In [22]:
json_file = open("facedetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("facedetector.h5")

In [23]:
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [24]:
def ef(image):
    img = load_img(image, grayscale = True)
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [25]:
!pip install matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
